#**[Task 1a] cleaning**



*   Clean the subject - removing the prefixes & alphanumeric stings.
*   Clean the body.
*   Filter the mails for language.


In [ ]:
#import modules
!pip install langdetect
import pandas as pd
import regex
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from langdetect import detect

     |████████████████████████████████| 983kB 5.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=7e6b0f899f018a2fe30e511c8a7c9e27433af91075dd01488c5ef749d41d9f0a
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
#load data
cleanedDataPath = "train.csv" # file path

df = pd.read_csv(cleanedDataPath, sep=";", encoding="utf8")
print("Number of overall Incidents: " + str(df.Id.size))
df = df.dropna(subset=['MailTextBody'])
df = df.dropna(subset=['MailSubject'])
print("Number of Incidents with body: " + str(df.Id.size))
print("columns: ", list(df.columns.values))
df = df.reset_index(drop=True)

df = df.sort_values(by='Id', ascending=False)


Number of overall Incidents: 2845
Number of Incidents with body: 2841
columns:  ['Unnamed: 0', 'Id', 'Impact', 'Urgency', 'IncidentType', 'ServiceProcessed', 'MailSubject', 'MailTextBody', 'ManualGroups']


In [ ]:
# clean subject
def cleanSub(sub):
  try:
    wordList = word_tokenize(sub)
    if len(wordList) <= 1:
      return sub
    if wordList[1] == ':':
      wordList.pop(0)
      wordList.pop(0)
    if wordList[-2] == ':':
      wordList.pop(-1)
      wordList.pop(-1)
      wordList.pop(-1)
    if any(ch == ':' for ch in str(wordList[0])):
      wordList.pop(0)
    if any(ch == ':' for ch in str(wordList[-1])):
      wordList.pop(-1)
    return ' '.join(word for word in wordList)
  except:
    return sub

# Clean text
def beautify_text(text):
    text = text.replace('\\r',' ')
    text = text.replace('\\n','')
    text = text.replace('\r',' ')
    text = text.replace('\n','')
    text = text.replace('  ',' ')
    text = text.lower()
    text = text.replace('&nbsp;','')
    return text.strip()

# detect language
def detectLanguage(x):
    try:
        return detect(x)
    except:
        pass


df['MailSubject'] = df['MailSubject'].apply(cleanSub)
df['MailTextBody'] = df['MailTextBody'].apply(beautify_text)
df['MailLanguage'] = df['MailTextBody'].apply(detectLanguage)
languageGroupedDf = df.groupby('MailLanguage')
pd.set_option('display.max_rows', None)
mailsByLanguage = languageGroupedDf.size().sort_values(ascending=False)
print(mailsByLanguage.nlargest(n=10))

df_de = df[(df['MailLanguage'] == 'de')]

pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_columns', None)
df_de.head()

# save cleaned file 
df_de.to_csv('train_clean.csv', index=False, sep=";")   # filtered data file to save

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
MailLanguage
de    2659
en     133
fr      34
it       4
so       2
nl       2
da       2
sv       1
no       1
hu       1
dtype: int64


In [ ]:
df_de.head()

,Unnamed: 0,Id,Impact,Urgency,IncidentType,ServiceProcessed,MailSubject,MailTextBody,ManualGroups,MailLanguage
0,3554,INC000010587669,4-Minor/Localized,2-High,Failure,EDA_S_BA_2FA,smart card blockiert,"lieber helpdesk sie haben mir den computer freigeschaltet, weil meine smartcard blockiert ist. ich bin nun in ausserholligen und möchte die smardcard gern deblockieren. was muss ich tun? bin jetzt...",NaN,de
1,3553,INC000010585556,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,Webcam,"liebe kolleginnen und kollegen ich möchte für susanne caseri eine webcam für videokoferenzen bestellen, die man am bildschirm befestigen kann. mit bestem dank und vielen grüsse",NaN,de
2,3552,INC000010585519,4-Minor/Localized,4-Low,Failure,EDA_S_Peripheriegeräte,IT Support heute Nachmittag,"liebe kolleginnen und kollegen es handelt sich um eine sitzung der deza-direktion heute um 16.30 uhr im sitzungszimmer 0 1 2 3 . darf ich sie gem. untenstehendem mail bitten, dass jemand vor ort s...",NaN,de
3,3549,INC000010582950,4-Minor/Localized,4-Low,Service Request,EDA_S_Benutzerunterstützung,Botschaft Warschau - Administratoren-Zugang für MS Teams,"sehr geehrte damen und herren, die botschaft in warschau plant am 18. november ein webinar zu organisieren. in diesem zusammenhang wurde ich intern gebeten, für mich einen administratoren-zugang f...",Benutzeranleitungen_Telefonie,de
4,3548,INC000010582919,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,Commande d'un écran,liebe kolleginnen und kollegen ich möchte für frédéric noirjean einen zweiten bildschirm bestellen. leider klappt die bestellung über egate nicht. ich kann keinen bildschirm auswählen. besten dank...,NaN,de


#**[Task 1b] Embedding (Doc2Vec)**


*   Get the embedding using Doc2Vec (tfhub)
*   Train ML classfier for predicting labels 

In [1]:
# import
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import regex
import re

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
cleanedDataPath = "train_clean.csv"

df = pd.read_csv(cleanedDataPath, sep=";", encoding="utf8")
print("Number of Incidents loaded: " + str(df.Id.size))

# df.head()
embedding_text = []
embedding_sub = []

for i, f in df.iterrows():
    if i == 0:
      embedding_text = embed([f.MailTextBody]).numpy()
      embedding_sub = embed([f.MailSubject]).numpy()
    else:
      embedding_text = np.concatenate((embedding_text, embed([f.MailTextBody]).numpy()), axis=0)
      embedding_sub = np.concatenate((embedding_sub, embed([f.MailSubject]).numpy()), axis=0)



**Training**

In [ ]:
# import
from sklearn import svm, preprocessing, feature_selection
from imblearn import under_sampling, over_sampling, combine
np.random.seed(0)

#lable
class_list = np.unique(df['ServiceProcessed'].dropna()).tolist()
y = df.apply(lambda x:  class_list.index(x['ServiceProcessed']),axis=1)
X = np.concatenate((embedding_sub, embedding_text), axis=1)

# basic feature selction
f_test, _ = feature_selection.f_classif(X, y)
f_test[np.isnan(f_test)] = 0
f_test /= np.max(f_test)
indexes = np.where(f_test > 0.2)
X = X[:, indexes]
X = X[:, 0, :]

# test train data gen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# normalization
X_scaler = preprocessing.StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# # supersampling
# cc = over_sampling.RandomOverSampler(random_state=0)
# # hybrid sampling - doen't work too few data points
# cc = combine.SMOTEENN(random_state=0)
# X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# clf = RandomForestClassifier()
# clf = xgb.XGBClassifier(),
# clf = RandomForestClassifier(),
clf = SVC(gamma='auto', decision_function_shape='ovo', class_weight='balanced')
clf.fit(X_resampled, y_resampled)
y_pred = clf.predict(X_test)

score = metrics.f1_score(y_test, y_pred, average='weighted')
score

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.45077501668169545

**Prediction**

In [ ]:
# training
cleanedDataPath = "train_clean.csv"
df = pd.read_csv(cleanedDataPath, sep=";", encoding="utf8")
print("Number of Incidents loaded: " + str(df.Id.size))

embedding_text = []
embedding_sub = []
for i, f in df.iterrows():
    if i == 0:
      embedding_text = embed([f.MailTextBody]).numpy()
      embedding_sub = embed([f.MailSubject]).numpy()
    else:
      embedding_text = np.concatenate((embedding_text, embed([f.MailTextBody]).numpy()), axis=0)
      embedding_sub = np.concatenate((embedding_sub, embed([f.MailSubject]).numpy()), axis=0)

x_train = np.concatenate((embedding_sub, embedding_text), axis=1)
class_list = np.unique(df['ServiceProcessed'].dropna()).tolist()
y_train = df.apply(lambda x:  class_list.index(x['ServiceProcessed']),axis=1)

x_scaler = preprocessing.StandardScaler().fit(x_train)
x_train = x_scaler.transform(x_train)

cc = over_sampling.RandomOverSampler(random_state=0)
x_resampled, y_resampled = cc.fit_resample(x_train, y_train)

clf = SVC(gamma='auto', decision_function_shape='ovo', class_weight='balanced')
clf.fit(x_resampled, y_resampled)

# prediction
cleanedDataPath = "test_reduced.csv"
df_test = pd.read_csv(cleanedDataPath, sep=";", encoding="utf8")
print("Number of Incidents loaded: " + str(df_test.Id.size))

embedding_text = []
embedding_sub = []

for i, f in df_test.iterrows():
    if i == 0:
      embedding_text = embed([f.MailTextBody]).numpy()
      embedding_sub = embed([f.MailSubject]).numpy()
    else:
      embedding_text = np.concatenate((embedding_text, embed([f.MailTextBody]).numpy()), axis=0)
      embedding_sub = np.concatenate((embedding_sub, embed([f.MailSubject]).numpy()), axis=0)

x_test = np.concatenate((embedding_sub, embedding_text), axis=1)
x_test = x_scaler.transform(x_test)
y_pred = clf.predict(x_test)

df_test['Predicted'] = y_pred
df_test['Predicted'] = df_test.apply(lambda x:  class_list[x['Predicted']],axis=1)
df_test = df_test[['Id', 'Predicted']]
df_test.to_csv('submission.csv', index=False)   # save to file